# 1_SPHEREx_LVF

# Build SPHEREx LVF Design

## Authors
- Yujin Yang, Woong-Seob Jeong (KASI SPHEREx Team)

## Goal
- Understand various photometric systems & filter transmission (broadband, narrowband)
- Understand linear variable filter
- Simulate LVF transmission
- Simulate SPHEREx wavelength map

## Recap: SPHEREx LVF definition

In [ ]:
# TODO: Include SPHEREx LVF specification here (image & table)

## Setting for this notebook

In [ ]:
# Setting for Jupyter notebook environment
# Display full output in Jupyter, not only last result.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"   # last or last_expr

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from astropy.table import Table

# Do not truncate outputs
np.set_printoptions(threshold=np.inf)

# Set default resolution for plots
mpl.rcParams["figure.dpi"] = 150

## 1. Filter Transmission Curves

### Broadband filters
* Examples for photometric system
    - Johnson, JHK, WISE, PanSTARRS, Subaru, Gaia DR2 
    - [Link of Spectlite package](https://speclite.readthedocs.io/en/latest/filters.html)
    - SDSS (https://www.astroml.org/_images/plot_sdss_filters_1.png)
    
![astroML plot](https://www.astroml.org/_images/plot_sdss_filters_1.png)

### Narrowband filters
- An example transmission curve for a narrowband filter

In [ ]:
T = Table.read('data/transmission_n419.csv')

plt.figure(figsize=(7,3))
plt.plot(T['Wavelength']*10, T['trans'])
plt.xlabel('wavelength [$\AA$]')
plt.ylabel('transmission')
plt.title('DECam N419 narrowband filter')

### Gaussian Approximation for narrowband filters

In [ ]:
from astropy.modeling.models import Gaussian1D

prof = Gaussian1D(amplitude=1, mean=2, stddev=0.2/2.35)
x = np.arange(1, 4, .01)
plt.plot(x, prof(x))
plt.xlabel('wavelength [um]')
plt.ylabel('transmission')

NameError: ignored

### Top Hat Function Approximation for narrowband filters

In [ ]:
def tophat_trans(x, center=0, fwhm=1, smoothness=0.2):

    from scipy.special import erf, erfc
    
    t_left  = erfc(+((2*(x-center)/fwhm)-1)/smoothness)/2 
    t_right = erfc(-((2*(x-center)/fwhm)+1)/smoothness)/2
    
    return (t_left*t_right)

In [ ]:
x = np.arange(1.8, 2.2, 0.001)
_ = plt.plot(x, tophat_trans(x, center=2, fwhm=0.1, smoothness=0.2), '-', alpha=0.5, label='smoothness=0.2')
_ = plt.plot(x, tophat_trans(x, center=2, fwhm=0.1, smoothness=0.6), '-', alpha=0.5, label='smoothness=0.6')
_ = plt.legend()
_ = plt.xlabel('wavelength [um]')
_ = plt.ylabel('transmission')

## 2. LVF Design
- Figure for LVF parameters
- Q. generate transmission as a function of (x, y)?
  - We will approximate the transmission curve as a Gaussian
  - `wl_lvf, tr_lvf = get_lvf_trans(xpos, ypos, index=index)`

### LVF with a fixed $\Delta\lambda$

In [ ]:
# Free parameters
R = 40        # Resolving power at each LVF point 
lmin = 1.00   # minimum wavelength [um]
dl = lmin / R # filter width [FWHM]
Nchan = 20    # Number of independent resolution elements with the LVF

ichan = np.arange(Nchan)
lcen = lmin + dl * ichan
width = np.zeros_like(ichan) + dl

In [ ]:
ichan
lcen
width

In [ ]:
# Different ways to print the data
for i, l, w in zip(ichan, lcen, width):
    print(f"{i:02d} {l:8.3f} {w:8.3f}")

T = Table([ichan, lcen], names=('ichan', 'lcen'))
T

print(np.c_[ichan, lcen, width])

In [ ]:
# Plot
wvec = np.linspace(0.5, 5.5, 5001)
plt.figure(figsize=(12,4))
for i, l, w in zip(ichan, lcen, width):
    prof = Gaussian1D(amplitude=1, mean=l, stddev=w/2.35)
    _ = plt.plot(wvec, prof(wvec))

_ = plt.xlim(0.9,2.0)
_ = plt.xlabel('wavelength [um]')
_ = plt.ylabel('transmission')

In [ ]:
plt.figure(figsize=(12,4))
for i, l, w in zip(ichan, lcen, width):
    trans = tophat_trans(wvec, center=l, fwhm=w)
    _ = plt.plot(wvec, trans)

_ = plt.xlim(0.9,2.0)
_ = plt.xlabel('wavelength [um]')
_ = plt.ylabel('transmission')

In [ ]:
# Spectral resolution
lcen / width
print(np.c_[ichan, lcen, lcen/width])

### LVF with a fixed resolving power ($R$)

In [ ]:
# Free parameters
# lmin, lmax, Nchan --> R
lmin = 1.00  # minimum wavelength [um]
Nchan = 5    # Number of independent resolution elements with the LVF
R = 40       # Resolving power at each LVF point

ichan = np.arange(Nchan)
lcen  = lmin * ( (2*R+1)/(2*R-1) )**ichan  # Why?
width = lcen / R

lcen
width

In [ ]:
# Free parameters
# lmin, lmax, Nchan --> R
lmin = 1.00  # minimum wavelength [um]
Nchan = 20   # Number of independent resolution elements with the LVF
R = 40       # Resolving power at each LVF point 

ichan = np.arange(Nchan)
lcen = lmin * ( (2*R+1)/(2*R-1) )**ichan
width = lcen / R
Rvec = np.zeros_like(lcen) + R

print(np.c_[ichan, lcen, width])

In [ ]:
# Plot in linear space
wvec = np.linspace(0.5,5.5,2001)
plt.figure(figsize=(12,4))
for i, l, w in zip(ichan, lcen, width):
    # prof = Gaussian1D(amplitude=1, mean=l, stddev=w/2.35)
    # _ = plt.plot(wvec, prof(wvec))
    trans = tophat_trans(wvec, center=l, fwhm=w)
    _ = plt.plot(wvec, trans)
    
_ = plt.xlim(0.9,1.9)
_ = plt.xlabel('wavelength [um]')
_ = plt.ylabel('transmission')

In [ ]:
# Plot in log space
wvec = np.linspace(0.5,5.5,5001)
plt.figure(figsize=(12,4))
for i, l, w in zip(ichan, lcen, width):
    trans = tophat_trans(wvec, center=l, fwhm=w)
    _ = plt.plot(wvec, trans)
    
_ = plt.xlim(0.9,1.9)
_ = plt.xscale('log')
_ = plt.xlabel('wavelength [um]')
_ = plt.ylabel('transmission')

## 3. LVF Implementation

### Mapping from detector pixels to central wavelengths
- 2D LVF wavelength map: (xpos, ypos) to $\lambda_C$
- Now we have to implement the LVF specification into a physical LVF. (**Engineers' regime**)
- Close interaction between astronomers and engineers are essential (like NISS & SPHEREx mission)
- Pointing accuray: spacing that we can sample
- 1D first, map ypos (in pixels) to lambda_C

In [ ]:
nx, ny = 1024, 2048

# place 0.5 channel at the beginning and end of the detector
ypos = (ny / Nchan) * (ichan+0.5)

(ny / Nchan) * (-0.5        + 0.5)   # edge: channel = -0.5  
(ny / Nchan) * (Nchan-1+0.5 + 0.5)   # edge: channel = Nchan-1 + 0.5

In [ ]:
def get_ichan(ypix):
    """
    Returns channel number for detector ypix
    """
    nx, ny = 1024, 2048
    return Nchan / ny * ypix - 0.5

lcen = lmin * ( (2*R+1)/(2*R-1) )**ichan

yvec = np.arange(2048)
_ = plt.plot(yvec, lmin * ( (2*R+1)/(2*R-1) )**get_ichan(yvec))
_ = plt.plot(ypos, lcen, 'o', alpha=0.7)
_ = plt.xlabel('Y [pixel]')
_ = plt.ylabel('Central Wavelength [um]')

### Telescope focal length & plate scale
 - plate scale [arcsec/mm] = $\frac{206265}{f [mm]}$
     - f = effective focal length
 - convert to pixel scale
     - pixscale = plate scale * detector pixel size
 - FOV: determined by optical performance, in this case, determined by the field stop (i.e., detector size)

- SPHEREx six H2RG (Hawaii-2RG infrared detectors)
    - Short wavelength: 2.6um cutoff
    - Long wavelength: 5.3um cutoff
    - 2048 x 2048 array, 18um x 18um pixels, HgCdTe technology

In [ ]:
# SPHEREx F-number = 3
206265 / (200*3) * 18e-3                   # pixel scale [arcsec]
206265 / (200*3) * 18e-3 * 2048 / 60 / 60  # FOV [deg]

### Make a LVF map & display

In [ ]:
xv = np.arange(nx)
yv = np.arange(ny)

xx, yy = np.meshgrid(xv, yv, indexing='xy')

wavemap = lmin * ( (2*R+1)/(2*R-1) )**get_ichan(yy)

wavemap.shape
xv.shape
xx.shape

In [ ]:
plt.imshow(xx, origin='lower', cmap='Blues')
plt.colorbar()
plt.title('X')

In [ ]:
plt.imshow(yy, origin='lower', cmap='Reds')
plt.colorbar()
plt.title('Y')

In [ ]:
plt.imshow(wavemap, origin='lower')
plt.colorbar()
plt.contour(wavemap, lcen, colors='k', linewidths=1, alpha=0.5)
plt.title('Wavelenth [$\mu$m]')

### Generate `get_lvf_trans()` function
- Write a Python function to get LVF transmission for a given detector position (xpos, ypos)
- `wl_lvf, tr_lvf = get_lvf_trans(x, y, band=1)`

In [ ]:
def get_lvf_trans(x, y, band=1):
    # `band` not used at the moments

    # Get R (hard-coded for now)
    # TODO: R should be determined from the band
    R = 40
    lmin = 1.0

    # get the central wavelenth
    wcen = lmin * ( (2*R+1)/(2*R-1) )**get_ichan(y)
    fwhm = wcen / R
    
    prof = Gaussian1D(amplitude=1, mean=wcen, stddev=fwhm/2.35)
    
    # wavelenth vector
    wl_lvf = np.linspace(wcen-fwhm*3, wcen+fwhm*3, 200)
    tr_lvf = prof(wl_lvf)
    
    return wl_lvf, tr_lvf

In [ ]:
xpos, ypos = 100, 2048/2
wl_lvf, tr_lvf = get_lvf_trans(xpos, ypos)
plt.plot(wl_lvf, tr_lvf)
_ = plt.xlabel('wavelength [um]')
_ = plt.ylabel('transmission')
_ = plt.title(f'Transmission at ({xpos}, {ypos})')

### Advanced: Smile Pattern
In reallity, it is not possible to apply coating exactly parallel to one of the axes. There will be small curvatures in LVF coatings, the so-called **smile pattern**. Let's simulate this patteren in an ad-hoc manner.

In [ ]:
x0 = nx/2
y0 = -10000
r2 = (xx-x0)**2 + (yy-y0)**2
yy_eff = np.sqrt(r2) + y0

plt.imshow(yy_eff, origin='lower', cmap='Reds')
plt.colorbar()
plt.title('Y')

In [ ]:
wavemap_smile = lmin * ( (2*R+1)/(2*R-1) )**get_ichan(yy_eff)
plt.imshow(wavemap_smile, origin='lower')
plt.colorbar()
levels = lcen
plt.contour(wavemap_smile, levels, colors='gray', linewidths=0.5, alpha=0.5)

# <span style='color:DarkSlateBlue'> Exercises </span>


## Exercise 1.1
- How many resolution elements (channels) do we need to cover from 0.75um to 5.0um with a constant R = 40?
- How many LVFs are required if each LVF can house 15 channels?

## Exercise 1.2
- For a given lmin = 1.0, lmax = 2.0, R = 100, how many independent channels can this LVF have?

## Exercise 1.3
- How many pointings does it take to cover the all sky?
- Assume a FOV = 3.5 degree

## Excercise 1.4 (SPHEREx) - advanced
- Repeat the above analysis for the current SPHEREx LVF design. Build the six SPHEREx LVF wavelength maps. You can ignore the `smiles`.

## Excercise 1.5 (Python)  - advanced
- update get_lvf_trans() function to have a profile function also as an argument
- `wl_lvf, tr_lvf = get_lvf_trans(x, y, profile='gaussian', band=1)`
- In fact, the `tophat_trans` function is broken or is destined to fail in a certain regime. Can you find it?

## Excercise 1.6 (7DT/7DS)
- Plot 7DT/7DS filter transmission curves
  - 40 medium band filters: $\Delta \lambda$ = 200A, $\lambda_{min}$ = 4,000, $\lambda_{max}$ = 8,000
- Re-design 7DS filter system so that the bandpass has a constant resolving power ($R$) instead

## Excercise 1.7 (7DT/7DS)
- Assume that we like to carry out the 7DS using LVF filters instead of 40 filters and 20 telescopes. How many spectral channels would be required if a single LVF will be used? Design this LVF wavelength map.